# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
# What model to download.
MODEL_NAME = 'prod_faster_rcnn_resnet50_coco'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [ ]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [ ]:
model_config="/home/wc-gpu/MasterThesis/models/research/object_detection/prod_faster_rcnn_resnet50_coco/model.config"

In [ ]:
from object_detection.protos import model_pb2
model_proto = model_pb2.DetectionModel()
with open(model_config, 'r') as myfile:
    config_string=myfile.read()
    text_format.Merge(config_string, model_proto)



In [ ]:
detection_model = model_builder.build(model_config=model_proto,
                                      is_training=False)

In [ ]:

# coord = tf.train.Coordinator()
# threads = tf.train.start_queue_runners(coord=coord)
# print('Coord start done...')

# preprocessed_images = []
# true_image_shapes = []

# image_lst = image_batch
# resized_image, true_image_shape = detection_model.preprocess(image_lst)
# preprocessed_images.append(resized_image)
# true_image_shapes.append(true_image_shape)

# rpn_features_to_crop, rett = detection_model._feature_extractor.extract_proposal_features(
#     resized_image, scope=detection_model.first_stage_feature_extractor_scope)

In [ ]:
# with detection_model.as_default():
#   od_graph_def = tf.GraphDef()
#   with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
#     serialized_graph = fid.read()
#     od_graph_def.ParseFromString(serialized_graph)
#     tf.import_graph_def(od_graph_def, name='')

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = '/Users/mbaltac/Home/MasterThesis/MasterThesis/Kickoff_MihaiBaltac/session_122923'
PATH_TO_TEST_IMAGES_DIR = '/home/wc-gpu/MasterThesis/session_data/122923_testing'

TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, f) for f in os.listdir(PATH_TO_TEST_IMAGES_DIR) if f.endswith(".jpg") ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, model):
    with tf.Graph().as_default() as graph:
#         tf.reset_default_graph()
   
        tf_image = tf.convert_to_tensor(np.expand_dims(image, 0), tf.float32)
        resized_image, true_image_shape = detection_model.preprocess(tf_image)
        rpn_features_to_crop, rett = model._feature_extractor.extract_proposal_features(
        resized_image, scope=detection_model.first_stage_feature_extractor_scope)
        init_op = tf.global_variables_initializer()

        with tf.Session() as sess:
            # Get handles to input and output tensors
            
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            print ("\n".join(all_tensor_names))
            tensor_dict = {}
            for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)
#             sess.run(init_op)
#             rpn_features_to_crop_out = sess.run(rpn_features_to_crop)
#             print (rpn_features_to_crop_out.shape, type(rpn_features_to_crop_out))
            
#             image_tensor = resized_image
#             print("\n".join([n.name for n in tf.get_default_graph().as_graph_def().node]))
            image_tensor = tf_image
            
            
            sess.run(init_op)
            rpn_features_to_crop_out = sess.run(rpn_features_to_crop)
            print (rpn_features_to_crop_out.shape, type(rpn_features_to_crop_out))
            
            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
        
    return output_dict, rpn_features_to_crop

In [ ]:
features = None
for image_path in sorted(TEST_IMAGE_PATHS):
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
   
    output_dict, features = run_inference_for_single_image(image_np, detection_model)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=2)
    slc = features[0][:,:,500]
    slc *= 255.0/slc.max()  

    img = Image.fromarray(slc)
    img = img.convert('RGB')

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image)

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(img)

    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    break